In [6]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [7]:
import fiona
import collections
import matplotlib.pyplot as plt
import pyproj
import rasterio
import rasterio.mask
from rasterio.plot import show
from rasterio import features
from sentinelsat import SentinelAPI, read_geojson, geojson_to_wkt
from datetime import date, timedelta
import geopandas
import os
from shapely import geometry
import numpy as np
import utilities
import pandas as pd
pd.set_option('display.max_columns', None)

In [11]:
# 18-21 Jul 2019
start_date = date(2019, 7, 12)
end_date = date(2019, 7, 13)

In [12]:
api = SentinelAPI('fernandeslouro', 'copernicospw', 'https://scihub.copernicus.eu/dhus')

In [13]:
for current_date in utilities.daterange(start_date, end_date -  timedelta(1)):
    # search by polygon, time, and SciHub query keywords
    footprint = geojson_to_wkt(read_geojson('quadrado-macao.geojson'))
    products = api.query(footprint,
                         date=(current_date, current_date + timedelta(1)),
                         platformname='Sentinel-2',
                         cloudcoverpercentage=(0, 30))
    products = api.to_dataframe(products)

In [16]:
list(products.title)

['S2A_MSIL2A_20190714T112121_N0213_R037_T29SPD_20190714T123155',
 'S2A_MSIL2A_20190714T112121_N0213_R037_T29TPE_20190714T123155',
 'S2A_MSIL2A_20190714T112121_N0213_R037_T29TNE_20190714T123155',
 'S2A_MSIL2A_20190714T112121_N0213_R037_T29SND_20190714T123155',
 'S2A_MSIL1C_20190714T112121_N0208_R037_T29TPE_20190714T114812',
 'S2A_MSIL1C_20190714T112121_N0208_R037_T29SPD_20190714T114812',
 'S2A_MSIL1C_20190714T112121_N0208_R037_T29TNE_20190714T114812',
 'S2A_MSIL1C_20190714T112121_N0208_R037_T29SND_20190714T114812']

In [15]:
# GeoJSON FeatureCollection containing footprints and metadata of the scenes
#api.to_geojson(products)

# GeoPandas GeoDataFrame with the metadata of the scenes and the footprints as geometries
#api.to_geodataframe(products)

# Get basic information about the product: its title, file size, MD5 sum, date, footprint and
# its download url
#api.get_product_odata(<product_id>)

# Get the product's full metadata available on the server
#api.get_product_odata(<product_id>, full=True)

In [21]:
# 18-21 Jul 2019
start_date = date(2017, 7, 20)
end_date = date(2017, 7, 30)
footprint = geojson_to_wkt(read_geojson('quadrado-cardigos.geojson'))
products = api.query(footprint,
                     date=(start_date, end_date),
                     platformname='Sentinel-2',
                     cloudcoverpercentage=(0, 30))
products_df = api.to_dataframe(products)

In [34]:
products_df

,title,link,link_alternative,link_icon,summary,ondemand,datatakesensingstart,ingestiondate,beginposition,endposition,orbitnumber,relativeorbitnumber,cloudcoverpercentage,filename,gmlfootprint,format,identifier,instrumentshortname,sensoroperationalmode,instrumentname,footprint,s2datatakeid,platformidentifier,orbitdirection,platformserialidentifier,processingbaseline,processinglevel,producttype,platformname,size,tileid,hv_order_tileid,uuid,level1cpdiidentifier,granuleidentifier,datastripidentifier,highprobacloudspercentage,mediumprobacloudspercentage,snowicepercentage,vegetationpercentage,waterpercentage,baresoilpercentage,lowprobacloudspercentage
a1a02e63-f416-4851-8fed-5fe95a685afd,S2B_MSIL1C_20170729T112119_N0205_R037_T29TNE_2...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,"Date: 2017-07-29T11:21:19.027Z, Instrument: MS...",false,2017-07-29 11:21:19.027,2017-07-30 01:50:47.834,2017-07-29 11:21:19.027,2017-07-29 11:21:19.027,2065,37,7.283800,S2B_MSIL1C_20170729T112119_N0205_R037_T29TNE_2...,"<gml:Polygon srsName=""http://www.opengis.net/g...",SAFE,S2B_MSIL1C_20170729T112119_N0205_R037_T29TNE_2...,MSI,INS-NOBS,Multi-Spectral Instrument,POLYGON ((-9.000236563891502 40.65085651577460...,GS2B_20170729T112119_002065_N02.05,2015-000A,DESCENDING,Sentinel-2B,02.05,Level-1C,S2MSI1C,Sentinel-2,816.15 MB,29TNE,TE29N,a1a02e63-f416-4851-8fed-5fe95a685afd,S2B_OPER_MSI_L1C_TL_SGS__20170729T163854_A0020...,S2B_OPER_MSI_L1C_TL_SGS__20170729T163854_A0020...,S2B_OPER_MSI_L1C_DS_SGS__20170729T163854_S2017...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
736b91a1-a263-4ef1-b1b7-c456ca8b3344,S2B_MSIL1C_20170729T112119_N0205_R037_T29SND_2...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,"Date: 2017-07-29T11:21:19.027Z, Instrument: MS...",false,2017-07-29 11:21:19.027,2017-07-30 01:49:05.372,2017-07-29 11:21:19.027,2017-07-29 11:21:19.027,2065,37,0.082100,S2B_MSIL1C_20170729T112119_N0205_R037_T29SND_2...,"<gml:Polygon srsName=""http://www.opengis.net/g...",SAFE,S2B_MSIL1C_20170729T112119_N0205_R037_T29SND_2...,MSI,INS-NOBS,Multi-Spectral Instrument,"POLYGON ((-9.000233454377252 39.7502679270046,...",GS2B_20170729T112119_002065_N02.05,2015-000A,DESCENDING,Sentinel-2B,02.05,Level-1C,S2MSI1C,Sentinel-2,836.81 MB,29SND,SD29N,736b91a1-a263-4ef1-b1b7-c456ca8b3344,S2B_OPER_MSI_L1C_TL_SGS__20170729T163854_A0020...,S2B_OPER_MSI_L1C_TL_SGS__20170729T163854_A0020...,S2B_OPER_MSI_L1C_DS_SGS__20170729T163854_S2017...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4cbbb813-4005-410d-a466-e6d4829fd0be,S2A_MSIL2A_20170724T112121_N0205_R037_T29SND_2...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,"Date: 2017-07-24T11:21:21.026Z, Instrument: MS...",false,NaT,2017-07-27 02:30:46.749,2017-07-24 11:21:21.026,2017-07-24 11:21:21.026,10902,37,0.245049,S2A_MSIL2A_20170724T112121_N0205_R037_T29SND_2...,"<gml:Polygon srsName=""http://www.opengis.net/g...",SAFE,S2A_MSIL2A_20170724T112121_N0205_R037_T29SND_2...,MSI,INS-NOBS,Multi-Spectral Instrument,"POLYGON ((-9.000233454377252 39.7502679270046,...",GS2A_20170724T112121_010902_N02.05,2015-000A,DESCENDING,Sentinel-2A,02.05,Level-2Ap,S2MSI2Ap,Sentinel-2,1.26 GB,NaN,NaN,4cbbb813-4005-410d-a466-e6d4829fd0be,NaN,NaN,NaN,0.134538,0.109903,0.00000,42.409058,0.385121,54.812016,1.246459
840f165f-2f91-4d34-98c5-5fbe828a6149,S2A_MSIL2A_20170724T112121_N0205_R037_T29TNE_2...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,"Date: 2017-07-24T11:21:21.026Z, Instrument: MS...",false,NaT,2017-07-27 02:00:45.952,2017-07-24 11:21:21.026,2017-07-24 11:21:21.026,10902,37,0.755469,S2A_MSIL2A_20170724T112121_N0205_R037_T29TNE_2...,"<gml:Polygon srsName=""http://www.opengis.net

In [22]:
products_df[['beginposition', 'size','format','cloudcoverpercentage','instrumentname']]

,beginposition,size,format,cloudcoverpercentage,instrumentname
a1a02e63-f416-4851-8fed-5fe95a685afd,2017-07-29 11:21:19.027,816.15 MB,SAFE,7.283800,Multi-Spectral Instrument
736b91a1-a263-4ef1-b1b7-c456ca8b3344,2017-07-29 11:21:19.027,836.81 MB,SAFE,0.082100,Multi-Spectral Instrument
4cbbb813-4005-410d-a466-e6d4829fd0be,2017-07-24 11:21:21.026,1.26 GB,SAFE,0.245049,Multi-Spectral Instrument
840f165f-2f91-4d34-98c5-5fbe828a6149,2017-07-24 11:21:21.026,1.21 GB,SAFE,0.755469,Multi-Spectral Instrument
72734452-f670-4e7c-8325-d6428222fa40,2017-07-24 11:21:21.026,843.47 MB,SAFE,0.000000,Multi-Spectral Instrument
72c8c5ce-aa7e-4af9-869f-54060405fe90,2017-07-24 11:21:21.026,830.74 MB,SAFE,0.046800,Multi-Spectral Instrument


In [27]:
list(products_df.index)
4cbbb813-4005-410d-a466-e6d4829fd0be
72734452-f670-4e7c-8325-d6428222fa40

['a1a02e63-f416-4851-8fed-5fe95a685afd',
 '736b91a1-a263-4ef1-b1b7-c456ca8b3344',
 '4cbbb813-4005-410d-a466-e6d4829fd0be',
 '840f165f-2f91-4d34-98c5-5fbe828a6149',
 '72734452-f670-4e7c-8325-d6428222fa40',
 '72c8c5ce-aa7e-4af9-869f-54060405fe90']

In [23]:
# download all results from the search 
api.download_all(products)

Downloading:   0%|          | 0.00/877M [00:00<?, ?B/s]
Downloading:   0%|          | 3.15M/877M [00:03<16:32, 881kB/s]
Downloading:   1%|          | 5.24M/877M [00:05<16:18, 891kB/s]/s]
Downloading:   1%|          | 7.34M/877M [00:08<16:54, 858kB/s]/s]
Downloading:   1%|          | 10.5M/877M [00:12<17:43, 815kB/s]]  
Downloading:   2%|▏         | 13.6M/877M [00:15<15:00, 959kB/s]/s]
Downloading:   2%|▏         | 17.8M/877M [00:19<13:29, 1.06MB/s]s]
Downloading:   2%|▏         | 21.0M/877M [00:21<12:59, 1.10MB/s]s]
Downloading:   3%|▎         | 23.1M/877M [00:24<14:10, 1.00MB/s]s]
Downloading:   3%|▎         | 25.2M/877M [00:26<15:56, 891kB/s] s]
Downloading:   3%|▎         | 26.2M/877M [00:28<18:15, 777kB/s]]  
Downloading:   3%|▎         | 27.3M/877M [00:29<18:04, 784kB/s]]
Downloading:   3%|▎         | 28.3M/877M [00:31<18:03, 783kB/s]]
Downloading:   3%|▎         | 30.4M/877M [00:34<18:53, 747kB/s]]
Downloading:   4%|▎         | 31.5M/877M [00:35<18:59, 742kB/s]]
Downloading:   4%

CancelledError: 

In [ ]:
!unzip S2B_MSIL2A_20210320T112109_N0214_R037_T29TNE_20210320T130843.zip
# Delete the zip

In [ ]:
def download_no_fail(product):
    try:
        output = api.download(p)
    except:
        download_no_fail(product)

***